In [ ]:
# | default_exp preprocess.dataloader

In [ ]:
# | export
import numpy as np
import pytorch_lightning as pl
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset

In [ ]:
# | export
import logging
import warnings

# Configure logging
logging.basicConfig(level=logging.WARNING)  # Change to DEBUG for more details

logger = logging.getLogger(__name__)

In [ ]:
# | hide


from fastcore.test import test_eq
from nbdev.showdoc import show_doc

In [ ]:
# | export
device = "cuda" if torch.cuda.is_available() else "cpu"
torch._dynamo.config.suppress_errors = True

In [ ]:
# | export


class TSRegressionDataset(Dataset):
    def __init__(self, df, in_features, out_features, window=1):
        super(TSRegressionDataset, self).__init__()

        self.data = df
        self.in_features = in_features
        self.out_features = out_features
        self.window = window

        self.groups = {uid: group.reset_index(drop=True) for uid, group in df.groupby("unique_id")}

    def __len__(self):
        assert self.in_features + self.out_features < len(
            self.data
        ), f"in_features + out_features should not be greater than series len {self.data.shape[0]}"

        return sum(
            [
                (len(g) - (self.in_features + self.out_features)) // self.window + 1
                for g in self.groups.values()
            ]
        )

    def __getitem__(self, idx):

        for uid, df in self.groups.items():
            max_idx = (len(df) - (self.in_features + self.out_features)) // self.window + 1

            if idx < max_idx:
                start = idx * self.window
                x = torch.tensor(
                    df["y"].iloc[start : start + self.in_features].values,
                    dtype=torch.float32,  # device=torch.device("cuda")
                )
                y = torch.tensor(
                    df["y"]
                    .iloc[start + self.in_features : start + self.in_features + self.out_features]
                    .values,
                    dtype=torch.float32,  # device=torch.device("cuda")
                )
                return x, y

            idx -= max_idx

In [ ]:
# | export


class TSDataLoader(pl.LightningDataModule):
    def __init__(self, df, in_features, out_features, window=1, batch_size=32):
        super(TSDataLoader, self).__init__()
        self.data = df
        self.in_features = in_features
        self.out_features = out_features
        self.window = window
        self.batch_size = batch_size

    def setup(self, stage=None):
        assert list(self.data.columns) == [
            "unique_id",
            "ds",
            "y",
        ], "Columns must be ['unique_id', 'ds', 'y']"

        self.data = self.data.sort_values(["unique_id", "ds"])

        # Train / Val / Test Split (70/15/15)
        train_sz = int(len(self.data) * 0.7)
        val_sz = int(len(self.data) * 0.15)

        self.train_df = self.data.iloc[:train_sz]
        self.val_df = self.data.iloc[train_sz : train_sz + val_sz]
        self.test_df = self.data.iloc[train_sz + val_sz :]

        self.train = TSRegressionDataset(
            self.train_df, self.in_features, self.out_features, self.window
        )
        self.val = TSRegressionDataset(
            self.val_df, self.in_features, self.out_features, self.window
        )
        self.test = TSRegressionDataset(
            self.test_df, self.in_features, self.out_features, self.window
        )

    def train_dataloader(self):
        return DataLoader(
            self.train,
            batch_size=self.batch_size,
            shuffle=True,
            pin_memory=True,
            num_workers=32,
            persistent_workers=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val,
            batch_size=self.batch_size,
            shuffle=False,
            pin_memory=True,
            num_workers=32,
            persistent_workers=True,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test,
            batch_size=self.batch_size,
            shuffle=False,
            pin_memory=True,
            num_workers=32,
            persistent_workers=True,
        )

In [ ]:
# | export
if __name__ == "__main__":
    from neuralforecast.utils import AirPassengersDF as df

    in_features, out_features, window, batch_sz = 12, 6, 1, 32
    ds = TSDataLoader(df, in_features, out_features, window, batch_sz)
    ds.setup()
    for features, labels in ds.train_dataloader():
        print(features.shape, labels.shape)

torch.Size([32, 12]) torch.Size([32, 6])
torch.Size([32, 12]) torch.Size([32, 6])
torch.Size([19, 12]) torch.Size([19, 6])


In [ ]:
# | export


class UnivariateTSDataset(Dataset):
    def __init__(self, windows):
        self.windows = windows

    def __len__(self):
        return len(self.windows)

    def __getitem__(self, idx):
        x, y = self.windows[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


class UnivariateTSDataModule(pl.LightningDataModule):
    def __init__(
        self,
        df,
        input_size,
        horizon,
        batch_size=32,
        num_workers=12,
        train_split=0.7,
        val_split=0.15,
        normalize=True,
        scaler_type="minmax",
        split_type="horizontal",  # New: "horizontal" or "vertical"
        step_size=1,  # New: Step size for sliding window
    ):
        """
        Args:
            df (pd.DataFrame): DataFrame with ['unique_id', 'ds', 'y'].
            input_size (int): Length of the lookback window.
            horizon (int): Length of the forecast horizon.
            batch_size (int): Batch size for DataLoader.
            train_split (float): Fraction for training.
            val_split (float): Fraction for validation (rest is test).
            normalize (bool): Whether to normalize data.
            scaler_type (str): 'minmax' or 'standard'.
            split_type (str): 'horizontal' (split within series) or 'vertical' (split by unique_id).
            step_size (int): Step size for sliding windows (1 = no skip, >1 = skip points).
        """
        super().__init__()
        self.df = df
        self.input_size = input_size
        self.horizon = horizon
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.train_split = train_split
        self.val_split = val_split
        self.normalize = normalize
        self.scaler_type = scaler_type
        self.split_type = split_type
        self.step_size = step_size
        self.scalers = {}

        # Validate splits
        if not 0 < train_split + val_split <= 1:
            raise ValueError("train_split + val_split must be between 0 and 1")
        if step_size < 1:
            raise ValueError("step_size must be >= 1")

    def setup(self, stage=None):
        grouped = self.df.groupby("unique_id")
        all_series = {unique_id: group["y"].values for unique_id, group in grouped}

        # Normalize series if requested
        if self.normalize:
            if self.scaler_type == "minmax":
                scaler_class = MinMaxScaler
            elif self.scaler_type == "standard":
                from sklearn.preprocessing import StandardScaler

                scaler_class = StandardScaler
            else:
                raise ValueError("scaler_type must be 'minmax' or 'standard'")

            normalized_series = {}
            for unique_id, series in all_series.items():
                scaler = scaler_class()
                normalized = scaler.fit_transform(series.reshape(-1, 1)).flatten()
                self.scalers[unique_id] = scaler
                normalized_series[unique_id] = normalized
        else:
            normalized_series = all_series

        train_windows, val_windows, test_windows = [], [], []

        if self.split_type == "horizontal":
            # Horizontal split: Split each series into train/val/test segments
            for unique_id, series in normalized_series.items():
                series_len = len(series)
                if series_len < self.input_size + self.horizon:
                    logger.warning(
                        f"{unique_id} - Series length {series_len} is too short for input_size={self.input_size} + horizon={self.horizon}"
                    )
                    continue

                max_idx = series_len - self.input_size - self.horizon + 1
                if max_idx <= 0:
                    logger.warning(f"{unique_id} - No valid windows possible")
                    continue

                # Adjust max_idx based on step_size
                num_windows = (max_idx - 1) // self.step_size + 1
                train_end = int(num_windows * self.train_split)
                val_end = train_end + int(num_windows * self.val_split)

                for j in range(0, max_idx, self.step_size):
                    if j + self.input_size + self.horizon > series_len:
                        break  # Ensure window doesn’t exceed series length
                    x = series[j : j + self.input_size]
                    y = series[j + self.input_size : j + self.input_size + self.horizon]
                    window = (x, y)
                    window_idx = j // self.step_size
                    if window_idx < train_end:
                        train_windows.append(window)
                    elif window_idx < val_end:
                        val_windows.append(window)
                    else:
                        test_windows.append(window)

        elif self.split_type == "vertical":
            # Vertical split: Split by unique_id
            unique_ids = list(normalized_series.keys())
            np.random.shuffle(unique_ids)  # Randomly shuffle series
            total_series = len(unique_ids)
            train_end = int(total_series * self.train_split)
            val_end = train_end + int(total_series * self.val_split)

            train_ids = unique_ids[:train_end]
            val_ids = unique_ids[train_end:val_end]
            test_ids = unique_ids[val_end:]

            for unique_id in train_ids:
                series = normalized_series[unique_id]
                series_len = len(series)
                if series_len < self.input_size + self.horizon:
                    logger.warning(f"{unique_id} - Series length {series_len} is too short")
                    continue

                max_idx = series_len - self.input_size - self.horizon + 1
                if max_idx <= 0:
                    continue

                for j in range(0, max_idx, self.step_size):
                    if j + self.input_size + self.horizon > series_len:
                        break
                    x = series[j : j + self.input_size]
                    y = series[j + self.input_size : j + self.input_size + self.horizon]
                    train_windows.append((x, y))

            for unique_id in val_ids:
                series = normalized_series[unique_id]
                series_len = len(series)
                if series_len < self.input_size + self.horizon:
                    logger.warning(f"{unique_id} - Series length {series_len} is too short")
                    continue

                max_idx = series_len - self.input_size - self.horizon + 1
                if max_idx <= 0:
                    continue

                for j in range(0, max_idx, self.step_size):
                    if j + self.input_size + self.horizon > series_len:
                        break
                    x = series[j : j + self.input_size]
                    y = series[j + self.input_size : j + self.input_size + self.horizon]
                    val_windows.append((x, y))

            for unique_id in test_ids:
                series = normalized_series[unique_id]
                series_len = len(series)
                if series_len < self.input_size + self.horizon:
                    logger.warning(f"{unique_id} - Series length {series_len} is too short")
                    continue

                max_idx = series_len - self.input_size - self.horizon + 1
                if max_idx <= 0:
                    continue

                for j in range(0, max_idx, self.step_size):
                    if j + self.input_size + self.horizon > series_len:
                        break
                    x = series[j : j + self.input_size]
                    y = series[j + self.input_size : j + self.input_size + self.horizon]
                    test_windows.append((x, y))

        else:
            raise ValueError("split_type must be 'horizontal' or 'vertical'")

        self.train_dataset = UnivariateTSDataset(train_windows)
        self.val_dataset = UnivariateTSDataset(val_windows)
        self.test_dataset = UnivariateTSDataset(test_windows)

        logger.info(
            f"Train windows: {len(train_windows)}, Val windows: {len(val_windows)}, Test windows: {len(test_windows)}"
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            pin_memory=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=True,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=True,
        )

    def inverse_transform(self, data, unique_id):
        if self.normalize and unique_id in self.scalers:
            return self.scalers[unique_id].inverse_transform(data.reshape(-1, 1)).flatten()
        return data